In [1]:
import os
import csv
import speech_recognition as sr
from evaluate import load
wer = load("wer")
from jiwer import wer




d:\study\codes\engineering_codes\research\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Function to read CSV file and process data
def read_csv_file(csv_file):
    data = []
    with open(csv_file, 'r') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            number_part = row['Number']
            text_part = row['Text'].strip()
            data.append((number_part, text_part))
    return data


In [3]:

# Function to convert wav file to text
def convert_wav_to_text(wav_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(wav_file) as source:
        audio = recognizer.record(source)
        try:
            return recognizer.recognize_google(audio)
        except sr.UnknownValueError:
            return ""
        except sr.RequestError as e:
            print(f"Could not request results; {e}")
            return ""


In [4]:

import string
def normalize_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text


In [5]:

# Function to process data and generate WER
def process_data_and_generate_wer(data,input_path, output_file):
    with open(output_file, 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(['Number', 'Expected Text', 'Generated Text', 'WER'])
        
        for number_part, expected_text in data:
            wav_file = os.path.join(input_path, f"{number_part}.wav")  # audio file path
            generated_text = convert_wav_to_text(wav_file)
            # error_rate = wer.compute(predictions=[normalize_text(generated_text)], references=[normalize_text(expected_text)])
            error_rate = wer(normalize_text(expected_text), normalize_text(generated_text))
            writer.writerow([number_part, expected_text, generated_text, error_rate])
            print(f"Processed {wav_file} - WER: {error_rate}")

In [6]:


    
# Set the path to your main dataset folder
dataset_folder = r"dataset\LibriSpeech\test-clean"
output_folder=r"outputs\libri_dataset_outputs"

for level1 in [f.name for f in os.scandir(dataset_folder) if f.is_dir()] :
    level1_name=level1
    level1=os.path.join(dataset_folder,level1)

    for level2 in [f.name for f in os.scandir(level1) if f.is_dir()]:
        level2_name=level2
        level2=os.path.join(level1,level2)
        level2=os.path.join(level2,'output')
        input_path=os.path.join(level2,'noise_added')

        #to check normal file wer 
        # input_path=level2
       

        for allfiles in [f.name for f in os.scandir(level2) ]:
            #accessing .csv file in that folder
            if allfiles.endswith('.csv'):
                csv_file=os.path.join(level2,allfiles)

                #interept the csv file and get number and text part
                data = read_csv_file(csv_file)

                #creating output file to save in outputs directory
                output_file_without_path=str(level1_name)+"-"+str(level2_name)+".csv"
                output_file=os.path.join(output_folder,output_file_without_path)

                #we need data [number part,text part] , input_path [which contains all .wav files] ,ouput_file [to store the final outputs]
                process_data_and_generate_wer(data, input_path,output_file)

Processed dataset\LibriSpeech\test-clean\260\123286\output\noise_added\260-123286-0000.wav - WER: 0.75
Processed dataset\LibriSpeech\test-clean\260\123286\output\noise_added\260-123286-0001.wav - WER: 1.0
Processed dataset\LibriSpeech\test-clean\260\123286\output\noise_added\260-123286-0002.wav - WER: 0.5862068965517241
Processed dataset\LibriSpeech\test-clean\260\123286\output\noise_added\260-123286-0003.wav - WER: 0.8823529411764706
Processed dataset\LibriSpeech\test-clean\260\123286\output\noise_added\260-123286-0004.wav - WER: 0.625
Processed dataset\LibriSpeech\test-clean\260\123286\output\noise_added\260-123286-0005.wav - WER: 1.0
Processed dataset\LibriSpeech\test-clean\260\123286\output\noise_added\260-123286-0006.wav - WER: 0.5
Processed dataset\LibriSpeech\test-clean\260\123286\output\noise_added\260-123286-0007.wav - WER: 0.7857142857142857
Processed dataset\LibriSpeech\test-clean\260\123286\output\noise_added\260-123286-0008.wav - WER: 1.0
Processed dataset\LibriSpeech\test